In [1]:
DOMAIN = 'test' # test, spouse, cdr

In [2]:
import sys
import os

if DOMAIN == 'cdr':
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 65


In [4]:
from snorkel.semantic import (get_left_tokens, get_right_tokens, 
                              get_between_tokens, get_sentence_tokens)
c = candidates[0]
# print c[0].get_parent()._asdict()['ner_tags']
# print get_left_tokens(c[0])
# print [getattr(t, 'words') for t in get_left_tokens(c[0])]

In [5]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [6]:
# c = sv.get_selected()
# print c

In [7]:
# hash(c)

In [8]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'cdr':
    user_lists = {}

In [9]:
# examples[0].display()

In [10]:
# examples[28].candidate[0].get_parent()

In [11]:
%time from snorkel.semantic import SemanticParser

CPU times: user 17 µs, sys: 2 µs, total: 19 µs
Wall time: 172 µs


In [12]:
%time sp = SemanticParser(candidate_class, user_lists, absorb=True)

Created grammar with 254 rules
CPU times: user 3.58 ms, sys: 4.2 ms, total: 7.79 ms
Wall time: 21 ms


In [13]:
%time results = sp.evaluate(examples, show_explanation=True, show_all=False, only=[])
results

Example 0: label True because True
Example 1: label True because True and True
Example 2: label True because False or True
Example 3: label True because not False
Example 4: label True because 'yes' equals 'yes'
Example 5: label True because 1 is equal to 1
Example 6: label True because 1 is equal to one
Example 7: label True because 1 is less than 2
Example 8: label True because 2 is less than or equal to 2
Example 9: label True because 2 > 1
Example 10: label True because 2 is at least 2
Example 11: label True because arg 1 is lowercase
Example 12: label True because arg 1 is upper case
Example 13: label True because arg 1 is capitalized
Example 14: label True because the word 'blueberry' starts with 'blue'
Example 15: label True because the word 'blueberry' ends with 'berry'
Example 16: label True because argument 1 is 'foo'
Example 17: label True because 'bar' is in 'foobarbaz'
Example 18: label True because the word 'foobarbaz' contains 'oobarba'
Example 19: label True because arg

,Correct,Incorrect,Redundant,Failed,Unknown
Example0,1,0,0,0,0
Example1,2,0,1,0,0
Example2,2,1,0,0,0
Example3,1,1,0,0,0
Example4,1,0,0,0,0
Example5,1,0,4,0,0
Example6,1,0,4,0,0
Example7,1,1,3,0,0
Example8,4,2,20,0,0
Example9,1,0,0,0,0


In [14]:
import numpy as np
print np.sum(results, axis=0)

Correct       145
Incorrect     146
Redundant    8194
Failed          0
Unknown         0
dtype: int64


In [15]:
# sp.grammar.print_chart(nested=True)

In [16]:
# sp.grammar.print_grammar()